# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-1. Применяем простейший локальный поиск.

В этой задаче Вам предлагается попробовать стандартную технику локального поиска (local search) в применении к задаче о сбалансированном разбиении графа. Мы будем рассматривать безвесовый вариант задачи с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* [Скачайте](http://mat.gsia.cmu.edu/COLOR/instances.html#XXMYC) файлы mycielX.col  (`for X in range(1,8)`).  (Если интересно, откуда такие графы берутся и чем интересны, см. конструкцию Зыкова—Мыцельского [здесь](https://docs.com/dainiak/3327).)
* Для каждого из графов найдите локальным поиском локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Ваша функция `basic_local_search` должна принимать на вход граф в формате, предоставляемом функцией `read_col_file`, и возвращать найденное локально минимальное разбиение просто как множество либо список вершин, лежащих в одной любой из двух компонент разбиения.

In [3]:
def read_col_file(filename: str) -> tuple:
    with open(filename, 'r') as file:
        vertices, edges = set(), set()
        for line in file:
            line = line.strip()
            if line.startswith('p'):
                vertices = set(range(1, int(line.split()[-2]) + 1))
            elif line.startswith('e'):
                edges.add(tuple(map(int, line.split()[-2:])))
        return (vertices, edges)

In [83]:
import random

def cut_cost(L: set, R: set, edges:set) -> int:
        ans = 0
        for edge in edges:
            v1, v2 = edge
            if (v1 in L and v2 in R) or (v1 in R and v2 in L):
                ans += 1
        return ans

def random_partition(vertices):
    L = set(random.sample(vertices, len(vertices) // 2))
    R = vertices - L
    
    return (L, R)

# в качестве 1-окрестности я рассматриваю разрезы
# отличающиеся от текущего в не более чем в 2 вершинах
# 2 -- если всего четное число вершин
# 1 -- иначе
def basic_local_search(graph: tuple) -> tuple:
    
    vertices, edges = graph
    n = len(vertices)
    L, R = random_partition(vertices)
    cur_cut_cost = cut_cost(L, R, edges)
    
    max_iterations = min(n * n, 1000)
    while True:
#         print(cur_cut_cost, end=' ')
        min_new_L = []
        min_new_R = []
        min_new_cut_cost = cur_cut_cost # типа бесконечность

        if len(L) == len(R):
            for v1 in L:
                for v2 in R:
                    L.remove(v1)
                    R.remove(v2)
                    L.add(v2)
                    R.add(v1)
                    
                    # <=, потому что если есть разрез поменьше
                    # то мы в итоге до него все равно дойдем
                    # а если нет, то этот переход нам все равно ничего не испортит
                    # а взамен мы, возможно, найдем через него разрез поменьше
                    cur_cut_cost = cut_cost(L, R, edges)
                    if cur_cut_cost <= min_new_cut_cost:
                        min_new_L = L
                        min_new_R = R
                        min_new_cut_cost = cur_cut_cost
                    
                    L.remove(v2)
                    R.remove(v1)
                    L.add(v1)
                    L.add(v2)
        else:
            # будем поддерживать |L| <= |R|
            for v in R:
                R.remove(v)
                L.add(v)
                
                cur_cut_cost = cut_cost(L, R, edges)
                if cur_cut_cost < min_new_cut_cost:
                    min_new_L = L
                    min_new_R = R
                    min_new_cut_cost = cur_cut_cost
                
                L.remove(v)
                R.add(v)
        
        if min_new_L == []:
            return (L, R)
        else:
            L = min_new_L
            R = min_new_R
            if (len(L) > len(R)):
                L, R = R, L
            cur_cut_cost = min_new_cut_cost

In [79]:
# Запустите эту ячейку, чтобы скачать файлы с графами
# Если все сломалось, пробуйте, пока не получится

import urllib.request

url_template = 'http://mat.gsia.cmu.edu/COLOR/instances/myciel{num}.col'


for i in range(3, 8):
    with open("myciel{num}.col".format(num=i), "w") as file:
        url = url_template.format(num=i)
        response = urllib.request.urlopen(url)
        print(response.read().decode('utf-8'), file=file)


In [ ]:
for i in range(3, 8):
    print("Myciel{num}:".format(num=i))
    
    vertices, edges = read_col_file('myciel{num}.col'.format(num=i))
    L, R = basic_local_search((vertices, edges))
    cut = cut_cost(L, R, edges)
    
    print(L, '\n', R, '\n', "cut cost = ", cut, '\n')